<a href="https://colab.research.google.com/github/om051105/Python/blob/main/TomatoLeafAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import kagglehub
import os

path = kagglehub.dataset_download("noulam/tomato")
print("Path:", path)
print(os.listdir(path))


100%|██████████| 743M/743M [00:07<00:00, 111MB/s]

Extracting files...


Path: /root/.cache/kagglehub/datasets/noulam/tomato/versions/1
['new plant diseases dataset(augmented)', 'New Plant Diseases Dataset(Augmented)']


In [4]:
BASE_DIR = None

for folder in os.listdir(path):
    full_path = os.path.join(path, folder)
    if os.path.isdir(full_path) and "Plant Diseases" in folder:
        BASE_DIR = full_path
        break

print("BASE_DIR:", BASE_DIR)
print("Contents:", os.listdir(BASE_DIR))


BASE_DIR: /root/.cache/kagglehub/datasets/noulam/tomato/versions/1/New Plant Diseases Dataset(Augmented)
Contents: ['valid', 'train']


In [5]:
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VALID_DIR = os.path.join(BASE_DIR, "valid")

print("TRAIN exists:", os.path.exists(TRAIN_DIR))
print("VALID exists:", os.path.exists(VALID_DIR))


TRAIN exists: True
VALID exists: True


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_data = valid_datagen.flow_from_directory(
    VALID_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)


Found 18345 images belonging to 10 classes.
Found 4585 images belonging to 10 classes.


In [7]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

NUM_CLASSES = 10  # tomato disease classes

base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False

mobilenet_model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation="softmax")
])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
mobilenet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [9]:
history_mobilenet = mobilenet_model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 256s 414ms/step - accuracy: 0.3917 - loss: 1.7903 - val_accuracy: 0.7385 - val_loss: 0.8159
Epoch 2/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 219s 382ms/step - accuracy: 0.7126 - loss: 0.8625 - val_accuracy: 0.8055 - val_loss: 0.6146
Epoch 3/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 217s 377ms/step - accuracy: 0.7769 - loss: 0.6781 - val_accuracy: 0.8181 - val_loss: 0.5463
Epoch 4/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 219s 381ms/step - accuracy: 0.8053 - loss: 0.5814 - val_accuracy: 0.8449 - val_loss: 0.4836
Epoch 5/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 215s 375ms/step - accuracy: 0.8215 - loss: 0.5360 - val_accuracy: 0.8561 - val_loss: 0.4594
Epoch 6/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 219s 381ms/step - accuracy: 0.8360 - loss: 0.4953 - val_accuracy: 0.8685 - val_loss: 0.4129
Epoch 7/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 217s 378ms/step - accuracy: 0.8504 - loss: 0.4509 - val_accuracy: 0.8702 - val_loss: 0.4034
Epoch 8/10
574/574 ━━━━━━━━━━━━━━━━━━━━ 218s 380ms/step - accuracy: 0.8542 -

In [10]:
mobilenet_model.save("tomato_disease_mobilenetv2.h5")
print("Model saved")


Model saved


In [11]:
from google.colab import files
files.download("tomato_disease_mobilenetv2.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
from google.colab import files
files.download("tomato_disease_mobilenetv2.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>